In [1]:
!guardrails hub install hub://reflex/valid_python

Installing hub://reflex/valid_python...
[=   ] Fetching manifestst
[    ] Downloading dependencies  Running command git clone --filter=blob:none --quiet https://github.com/reflex-dev/valid_python_validator.git /private/var/folders/c8/jqt82fpx785dpwpp36ljkgm40000gn/T/pip-req-build-wnm6yigd
[  ==] Downloading dependencies
[    ] Running post-install setup
✅Successfully installed reflex/valid_python!


Import validator:
from guardrails.hub import ValidPython

Get more info:
https://hub.guardrailsai.com/validator/reflex/valid_python



# Generating Bug Free Leetcode Solutions

!!! note
    To download this tutorial as a Jupyter notebook, click [here](https://github.com/guardrails-ai/guardrails/blob/main/docs/examples/bug_free_python_code.ipynb).

In this example, we want to solve String Maniuplation leetcode problems such that the code is bug free.

We make the assumption that:

1. We don't need any external libraries that are not already installed in the environment.
2. We are able to execute the code in the environment.

## Objective

We want to generate bug-free code for solving leetcode problems. In this example, we don't account for semantic bugs, only for syntactic bugs.

In short, we want to make sure that the code can be executed without any errors.

## Step 1: Generating `RAIL` Spec

Ordinarily, we could create a separate `RAIL` spec in a file. However, for the sake of this example, we will generate the `RAIL` spec in the notebook as a string.  We will also show the same RAIL spec in a code-first format using a Pydantic model.

XML option:

In [2]:
rail_str = """
<rail version="0.1">

<output>
    <string
        name="python_code"
        format="bug-free-python"
        on-fail-bug-free-python="reask"
    />
</output>


<prompt>
Given the following high level leetcode problem description, write a short Python code snippet that solves the problem.

Problem Description:
${leetcode_problem}

${gr.complete_json_suffix}</prompt>

</rail>
"""

Pydantic model option:

In [3]:
from pydantic import BaseModel, Field
from guardrails.hub import ValidPython

prompt = """
Given the following high level leetcode problem description, write a short Python code snippet that solves the problem.

Problem Description:
${leetcode_problem}

${gr.complete_json_suffix}"""

class BugFreePythonCode(BaseModel):
    python_code: str = Field(validators=[ValidPython(on_fail="reask")])

    class Config:
        arbitrary_types_allowed = True

## Step 2: Create a `Guard` object with the RAIL Spec

We create a `gd.Guard` object that will check, validate and correct the generated code. This object:

1. Enforces the quality criteria specified in the RAIL spec (i.e. bug free code).
2. Takes corrective action when the quality criteria are not met (i.e. reasking the LLM).
3. Compiles the schema and type info from the RAIL spec and adds it to the prompt.

In [4]:
import guardrails as gd

from rich import print

From XML:

In [5]:
guard = gd.Guard.from_rail_string(rail_str)

/Users/zaydsimjee/workspace/guardrails/docs/examples/.venv/lib/python3.10/site-packages/guardrails/validatorsattr.py:307: UserWarning: Validator bug-free-python is not installed!
  warnings.warn(f"Validator {validator_name} is not installed!")


Or from the pydantic model:

In [6]:
guard = gd.Guard.from_pydantic(output_class=BugFreePythonCode, prompt=prompt)

/Users/zaydsimjee/workspace/guardrails/docs/examples/.venv/lib/python3.10/site-packages/guardrails/validators/__init__.py:51: FutureWarning: 
    Importing validators from `guardrails.validators` is deprecated.
    All validators are now available in the Guardrails Hub. Please install
    and import them from the hub instead. All validators will be
    removed from this module in the next major release.

    Install with: `guardrails hub install hub://<namespace>/<validator_name>`
    Import as: from guardrails.hub import `ValidatorName`
    
  warn(


The `Guard` object compiles the output schema and adds it to the prompt. We can see the final prompt below:

In [7]:
print(guard.base_prompt)

/var/folders/c8/jqt82fpx785dpwpp36ljkgm40000gn/T/ipykernel_83057/3983563700.py:1: DeprecationWarning: 'Guard.base_prompt' is deprecated and will be removed in versions 0.5.x and beyond. Use 'Guard.history.last.prompt' instead.
  print(guard.base_prompt)


Given the following high level leetcode problem description, write a short Python code snippet that solves the 
problem.

Problem Description:
${leetcode_problem}


Given below is XML that describes the information to extract from this document and the tags to extract it into.

<output>
    <string name="python_code" format="reflex/valid_python"/>
</output>


ONLY return a valid JSON object (no other text is necessary), where the key of the field in JSON is the `name` 
attribute of the corresponding XML, and the value is of the type specified by the corresponding XML's tag. The JSON
MUST conform to the XML format, including any types and format requests e.g. requests for lists, objects and 
specific types. Be correct and concise. If you are unsure anywhere, enter `null`.

Here are examples of simple (XML, JSON) pairs that show the expected behavior:
- `<string name='foo' format='two-words lower-case' />` => `{'foo': 'example one'}`
- `<list name='bar'><string format='upper-case' /></list>` => `{"bar": ['STRING ONE', 'STRING TWO', etc.]}`
- `<object name='baz'><string name="foo" format="capitalize two-words" /><integer name="index" format="1-indexed" 
/></object>` => `{'baz': {'foo': 'Some String', 'index': 1}}`

## Step 3: Wrap the LLM API call with `Guard`

In [8]:
import openai

leetcode_problem = """
Given a string s, find the longest palindromic substring in s. You may assume that the maximum length of s is 1000.
"""

response = guard(
    openai.chat.completions.create,
    prompt_params={"leetcode_problem": leetcode_problem},
    max_tokens=2048,
    temperature=0,
)

Running the cell above returns:
1. The raw LLM text output as a single string.
2. A dictionary where the key is `python_code` and the value is the generated code.

In [9]:
print(response.validated_output)

{
    'python_code': 'class Solution:\n    def longestPalindrome(self, s: str) -> str:\n        if len(s) == 0:\n    
return ""\n        start = 0\n        end = 0\n        for i in range(len(s)):\n            len1 = 
self.expandAroundCenter(s, i, i)\n            len2 = self.expandAroundCenter(s, i, i + 1)\n            max_len = 
max(len1, len2)\n            if max_len > end - start:\n                start = i - (max_len - 1) // 2\n           
end = i + max_len // 2\n        return s[start:end + 1]\n\n    def expandAroundCenter(self, s: str, left: int, 
right: int) -> int:\n        while left >= 0 and right < len(s) and s[left] == s[right]:\n            left -= 1\n  
right += 1\n        return right - left - 1'
}

Here's the generated code:

In [10]:
if response.validated_output is not None:
    print(response.validated_output["python_code"])
elif response.error is not None:
    print(response.error)

class Solution:
    def longestPalindrome(self, s: str) -> str:
        if len(s) == 0:
            return ""
        start = 0
        end = 0
        for i in range(len(s)):
            len1 = self.expandAroundCenter(s, i, i)
            len2 = self.expandAroundCenter(s, i, i + 1)
            max_len = max(len1, len2)
            if max_len > end - start:
                start = i - (max_len - 1) // 2
                end = i + max_len // 2
        return s

    def expandAroundCenter(self, s: str, left: int, right: int) -> int:
        while left >= 0 and right < len(s) and s == s:
            left -= 1
            right += 1
        return right - left - 1

We can confirm that the code is bug free by executing the code in the environment.

In [11]:
try:
    exec(response.validated_output["python_code"])
    print("Success!")
except Exception as e:
    print("Failed!")

Success!